In [2]:
rm(list=ls())
suppressMessages(library(DoubleML))
suppressMessages(library(mlr3))
suppressMessages(library(mlr3learners))
suppressMessages(library(tidyverse))
suppressMessages(library(haven))
suppressMessages(library(MASS))
suppressMessages(library(sjlabelled))
suppressMessages(library(varhandle))
suppressMessages(library(stargazer))
suppressMessages(library(xtable))

lgr::get_logger("mlr3")$set_threshold("warn")

# Machine Learning Estimation Functions

In [3]:
# Random Forest Estimation Function
est_forest <- function(obj_dml_data) {
    
    learner <- lrn("regr.ranger", num.trees=100, min.node.size=2, max.depth=5)
    ml_m <- learner$clone()
    ml_g <- learner$clone()
    
    obj_dml_plr  <- DoubleMLPLR$new(obj_dml_data, ml_g, ml_m)
    
    obj_dml_plr$fit()
    theta = rbind(obj_dml_plr$coef,obj_dml_plr$se)
    return(theta)
}

# LASSO Estimation Function
est_lasso <- function(obj_dml_data) {
    
    learner <- lrn("regr.cv_glmnet", s="lambda.min")
    ml_g <- learner$clone()
    ml_m <- learner$clone()

    obj_dml_plr  <- DoubleMLPLR$new(obj_dml_data, ml_g, ml_m)

    obj_dml_plr$fit()
    theta = rbind(obj_dml_plr$coef, obj_dml_plr$se)
    return(theta)
}

# Regression Tree Estimation
est_rt   <- function(obj_dml_data){

    learner <- lrn("regr.rpart")
    ml_g <- learner$clone()
    ml_m <- learner$clone()
    
    obj_dml_plr = DoubleMLPLR$new(obj_dml_data, ml_g, ml_m)

    param_grid = list(
    "ml_g" = paradox::ParamSet$new(list(
             paradox::ParamDbl$new("cp", lower = 0.01, upper = 0.02),
             paradox::ParamInt$new("minsplit", lower = 1, upper = 2))),
     "ml_m" = paradox::ParamSet$new(list(
             paradox::ParamDbl$new("cp", lower = 0.01, upper = 0.02),
          paradox::ParamInt$new("minsplit", lower = 1, upper = 2))))

    # minimum requirements for tune_settings
    tune_settings = list(
    terminator = mlr3tuning::trm("evals", n_evals = 5),
    algorithm = mlr3tuning::tnr("grid_search", resolution = 5))
    suppressMessages(obj_dml_plr$tune(param_set = param_grid, tune_settings = tune_settings))
    suppressMessages(obj_dml_plr$fit())
    
    theta = rbind(obj_dml_plr$coef, obj_dml_plr$se)
    return(theta)

}


In [4]:
#df<-pdata.frame(panel_data_set,index=c("village",))

###function input is a df of control variables
lasso_data_generator<-function(df){
    len<-ncol(df)
##each variable is multiplied by all other variables so includes
    repeat_remover<-0
    for (variable in 1:len){
        for (loop in 1:len){
            if(loop + repeat_remover <=len){
            df=cbind(df,(df[,variable])*(df[,loop+repeat_remover]))
 
            }
        }
    repeat_remover<- repeat_remover+1
    }
 return(df)
}

In [5]:
##load in dataset which consists of the baseline dataset of Somville and Vandewallle(2018) with their variables creations and manipulations
df<-read_dta("merged_and_edited_stata.dta")

In [6]:
set.seed(4)
Y_var= c("balance_final", "balance_average", "exp_freq", "exp_tempt", "sav_cash", "sav_nonbcsa_all", "sav_bcsa_all")
control=c("cat_bcsa_open", "female", "sc", "obc", "fc", "readwrite_dum", "married", "age", "emp_agr", "emp_nonagr", "self_agr", "self_nonagr", "land_amount", "dwelling_katcha", "account_total", "shg_total", "savings_decision", "trust_bank_bcsa", "impatient", "bcsa_distance","village")
regression_df<-data.frame(matrix(ncol=2,nrow=442))
colnames(regression_df)<-c("y","d")
regression_df["d"]<-df["random_account"]
theta<-data.frame(matrix(ncol=7,nrow=6))
colnames(theta)<-c("balance_final", "balance_average", "exp_freq", "exp_tempt", "sav_cash", "sav_nonbcsa_all", "sav_bcsa_all")

regressed<-c()
for (var in Y_var){
    regression_df["y"]<-df[,var]  
    test_data<-subset(df,select=c(cat_bcsa_open, female, sc, obc, fc, readwrite_dum, married, age, emp_agr, emp_nonagr, self_agr, self_nonagr, land_amount, dwelling_katcha, account_total, shg_total, savings_decision, trust_bank_bcsa, impatient, bcsa_distance, village))
    regressed<-c()
    regressed<-cbind(regression_df,test_data)
    regressed<-remove_all_labels(regressed)
        if (var != "balance_final"){
            if (var != "balance_average"){
                regressed<- regressed %>% drop_na(y)

            }
        }
        
    regressed<-cbind(regressed, to.dummy(regressed$village, "dummy"))

    obj_dml_data <- double_ml_data_from_data_frame(regressed, y_col = "y", d_cols = "d")

    
    theta_forest <- est_forest(obj_dml_data)  
    theta_rt     <- est_rt(obj_dml_data)
    
    ###implement special data process for lasso
    test_data<-lasso_data_generator(test_data)
    regressed<-c()
    regressed<-cbind(regression_df,test_data)
    regressed<-remove_all_labels(regressed)
           if (var != "balance_final"){
            if (var != "balance_average"){
                regressed<- regressed %>% drop_na(y)
            }
        }
        
    regressed<-cbind(regressed, to.dummy(regressed$village, "dummy"))
    
    obj_dml_data <- double_ml_data_from_data_frame(regressed, y_col = "y", d_cols = "d")
    theta_lasso  <- est_lasso(obj_dml_data)
    theta[,var] <- rbind(theta_lasso, theta_forest, theta_rt)
    regressed<-c()
    
}

INFO  [14:33:01.202] [bbotk] Starting to optimize 2 parameter(s) with '<OptimizerGridSearch>' and '<TerminatorEvals> [n_evals=5, k=0]' 
INFO  [14:33:01.249] [bbotk] Evaluating 1 configuration(s) 
INFO  [14:33:01.453] [bbotk] Result of batch 1: 
INFO  [14:33:01.453] [bbotk]      cp minsplit regr.mse runtime_learners                                uhash 
INFO  [14:33:01.453] [bbotk]  0.0125        1   807672             0.05 5d37882c-83c9-49ea-90ce-dcbb38fd95a6 
INFO  [14:33:01.453] [bbotk] Evaluating 1 configuration(s) 
INFO  [14:33:01.645] [bbotk] Result of batch 2: 
INFO  [14:33:01.647] [bbotk]    cp minsplit regr.mse runtime_learners                                uhash 
INFO  [14:33:01.647] [bbotk]  0.02        1   807672             0.05 2b0f284d-2b3b-4709-b08b-c9aa111deaa6 
INFO  [14:33:01.649] [bbotk] Evaluating 1 configuration(s) 
INFO  [14:33:01.817] [bbotk] Result of batch 3: 
INFO  [14:33:01.817] [bbotk]     cp minsplit regr.mse runtime_learners                               

In [7]:
print(xtable(theta, type = "latex"), file = "filename2.tex")

In [8]:
print(theta)

          d         d         d        d         d        d         d
1 475.14800 306.51215 -423.8974 33.06187 -214.9938 440.8152 1018.9708
2  74.26299  68.06494  213.4598 64.66113  486.4943 562.7800  573.6420
3 449.10704 310.97806 -313.3232 52.29963 -349.1063 306.0310  916.9577
4  77.06334  65.43198  201.9578 60.84301  475.2836 531.9392  502.8722
5 477.45218 312.32533 -391.8916 30.98971 -249.1225 434.2719  845.0844
6  82.33663  73.21779  221.9841 67.22923  487.1405 561.8466  569.6320
